**This Jupyter notebook includes the numerical methods used in the project and coded myself. The numerical mothods include: Vector Autoregression (class VAR), Augmented Dickey Fuller Test (class ADF). Also, here I introduce a more accurate critical values of 1%, 5% and 10% for the Augmented Dickey Fuller test to use. The aim of this notebook is to choose the future pair for the cointegration trading strategy.**

In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from scipy.stats import norm
from tabulate import tabulate

## Introduction of Functions and Classes

**These functions and classes are packed in the numerical_methods.py file for the use of other Jupyter notebooks**

### Critical Value

The function **CriticalValue(level, T)** aims to give more accurate critical values for the Augmented Dickey Fuller test. 

**Inputs**:

- level: can take values of '1%', '5%' and '10%'
- T: the number of observed values

**Output**:

- cvalue: the critical value for a certain level adjusted by the number of samples

The critical values are calculated by:

$$ \beta_{\infty} + \beta_{1}/T + \beta_{2}/T^2 + \beta_{3}/T^3$$

where $T$ is the number of samples; $\beta_{\infty}$, $\beta_{1}$, $\beta_{2}$ and $\beta_{3}$ are parameters that can be found in "Table 2" of the paper *Critical Values for Cointegration Tests* by James G.MacKinnon. The formula above is also explained in the paper. For the use of this project, the parameters are of the case of "N = 2" (two variables) and "With Constant" in the Table. $\beta_{3}$ has no value in this case.

In [2]:
def CriticalValue(level, T):
    """
    Critical values of Table 2 in the artical 'Critical Values for Cointegration Tests' from James G.MacKinnon
    Condition of:
        N=2: 2 predictors(variables) 
        With constant
    Adjusted for the finite sample size: T
    """
    if level=='1%':
        cvalue = -3.89644 - 10.9519/T - 22.527/T**2
    elif level=='5%':
        cvalue = -3.33613 - 6.1101/T - 6.823/T**2
    elif level=='10%':
        cvalue = -3.04445 - 4.2412/T - 2.720/T**2
    else:
        cvalue = False
    return cvalue

### Regression
The function **regress(X, Y)** is for the calculation of the OLS regression in matrix form. 

**Inputs**:

- X: the explanatory matrix in nparray which contains a row of 1 representing the constant and the explanatory variable(s). 
- Y: the dependent matrix or vector in nparray which contains the dependent variable(s).

**Output**:

- res: a dictionary which contains:
    - 'coef': coefficients of the regression
    - 'std err': standard error for each coefficient
    - 't-stats': T statistic value for each coefficient
    - 'p-value': p value for each coefficient
    - 'AIC': Akaike Information Criterion value (only applied in VAR, set to be None in simple regression)
    - 'BIC': Bayesian Information Criterion value (only applied in VAR, set to be None in simple regression)
    - 'resid': residual matrix
    
This function is used in the construction of both VAR and ADF classes.

In [3]:
def regress(X, Y):
    """
    OLS regression with:
        X: explanatory Matrix
        Y: dependent Matrix/Vector
    Return: 
        a dictionary contains: coefficient, Standard error, T stats, p value, AIC, BIC
    """
    # Number of X, Y variables
    Nx = X.shape[0]
    if Y.ndim == 1:
        Ny = Y.ndim
    else:
        Ny = Y.shape[0]
    
    # Number of observations
    Nobs = X.shape[1]
    
    # calculate the coefficient
    Beta = Y@X.T@np.linalg.pinv((X@X.T))
    
    # Calculate disturbance matrix
    Resid = Y - Beta@X
    
    # Estimator of the residual covariance matrix
    Sigma = (Resid@Resid.T)/Nobs
    
    # Standard error of beta coefficient
    Iinv = np.kron(np.linalg.pinv(X@X.T), Sigma)
    std_err = np.sqrt(np.diagonal(Iinv)).reshape(Nx, Ny).T
    
    # T-statistics
    T = np.divide(Beta, std_err)
    
    # P-value
    p_value = 2 * norm.sf(np.abs(T))
    
    if not isinstance(Sigma, np.ndarray):
        AIC = BIC = None
    else:
        k = Nx*Ny
        # AIC
        AIC = np.log(np.linalg.det(Sigma)) + 2 * k / Nobs
        # BIC
        BIC = np.log(np.linalg.det(Sigma)) + k / Nobs *np.log(Nobs)
    
    res = {
        'coef': Beta,
        'std err': std_err,
        't-stats': T,
        'p-value': p_value,
        'AIC': AIC,
        'BIC': BIC,
        'resid': Resid
    }
    return res

### Explanatory Matrix
The function **explanatory_matrix(X, p)** is for the construction of the explanatory matrix of the OLS regression, which is an input for the regress(X, Y) function.

**Inputs**:

- X: (part of) the explanatory variables in pd.DataFrame
- p: number of lags

**Output**:

- Z: a matrix in nparray which contains: a row of 1 representing constant, explanatory variables with lag p

When lag is 0, it simply add a row of 1 to the original explanatory variables. When p > 0 as well as VAR case, it creates more explanatory variables.

In [4]:
def explanatory_matrix(X, p):
    """
    X: explanatory variable in pd.DataFrame
    p: number of lag
    
    return: an explanatory matrix with lag p
    """
    N = len(X)
    # Form the explanatory matrix  
    if isinstance(X, pd.Series):
        Z = np.array(np.ones(N-p))
        Z = np.vstack((Z, X[p:].T))
        return Z
    else:
        Z = np.array(np.ones(N-p))
        if p==0:
            # simply add a row of 1
            Z = np.vstack((Z, X.values.T))
        else:
            for i in range(1, p+1):
                # form new variables
                Z = np.vstack((Z, X[p-i:N-i].values.T))
        return Z

### Vector Autoregression

The class **VAR(Y, p)** construct an instance of the vector autoregression estimation.

**Inputs**:

- Y: the variables for the VAR in pd.DataFrame
- p: number of lags

**Methods**:

- estimation(): return the OLS regression result in pd.DataFrame, the result contains information of the coefficients, standard error, T statistic value and p value for each variable


- AIC_BIC(prt): 
    - prt = True: default case, print a table of lag, AIC value and BIC value
    - prt = False: return a list of lag, AIC value and BIC value
    
    
- Stability(prt):
    - prt = True: default case, print a table of eigenvalues of relationship matrices, modulus of the eigenvalues and whether it is stable for each eigenvalue for every lag from 1 up to p 
    - prt = False: return a overall stability result (True if all eigenvalues are stable)


- correlation(): return the correlation between each pair of variables and the corresponding correlation value in descending order


This class aims to find the best lag and the best pair of futures for the cointegration trading strategy.

In [5]:
class VAR():
    def __init__(self, Y, p):
        """
        VAR with lag p in the matrix form
        Inputs:
            Y: A DataFrame containing the variables in time series format.
            p: number of lags
        """
        self.Y = Y
        self.p = p
        # number of observations
        N = len(self.Y)
        Nobs = N - self.p
        # column names
        self.col_names = list(self.Y.columns)
        # number of variables
        Nvar = len(self.Y.columns)
        # Form the explanatory matrix
        Xmat = explanatory_matrix(self.Y, self.p)
        # Form the dependent matrix
        Ymat = self.Y[self.p:].values.T
        # OLS result
        self.res = regress(Xmat, Ymat)
        
        # Stability condition
        self.Eigenvalues = dict.fromkeys(range(1, self.p+1))
        for i in range(1, self.p+1):
            Bp = self.res['coef'][:,1 + (i-1) * Nvar: 1 + i*Nvar]
            eigenvalues, _ = np.linalg.eig(Bp)
            self.Eigenvalues[i] = abs(eigenvalues)
            
    def estimation(self):
        """
        Giving a table of Estimation, includes:
            Coefficients beta
            Standard error
            T statistics
            P value
        for the VAR of each variable
        """
        Beta = self.res['coef']
        std_err = self.res['std err']
        T = self.res['t-stats']
        p_value = self.res['p-value']
        
        table_header = ['Variable', 'Stats', 'Const']
        table_data = []
        for i in range(1, self.p + 1):
            table_header += [key + '(-' + str(i)+ ')' for key in self.col_names]
        for i in range(len(self.col_names)):
            table_data.append([self.col_names[i] , 'Estimates'] + ['{:.4f}'.format(item) for item in Beta[i]])
            table_data.append([self.col_names[i] , 'Std err'] + ['{:.4f}'.format(item) for item in std_err[i]])
            table_data.append([self.col_names[i] , 't-stats'] + ['{:.4f}'.format(item) for item in T[i]])
            table_data.append([self.col_names[i] , 'p-value'] + ['{:.4f}'.format(item) for item in p_value[i]])
        estimation_result = pd.DataFrame(table_data, columns=table_header)
        return estimation_result
        
    def AIC_BIC(self, prt = True):
        """
        prt = True: Giving a table of AIC and BIC for the lag p
        prt = False: return the lag p, AIC and BIC values
        """
        AIC = self.res['AIC']
        BIC = self.res['BIC']
        
        if prt:
            table_header = ['Lag', 'AIC', 'BIC']
            table_data = [[str(self.p), '{:.4f}'.format(AIC), '{:.4f}'.format(BIC)]]
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        else:
            return [self.p, AIC, BIC]
    
    def Stability(self, prt = True):
        """
        prt = True: Giving a table of the eigenvalues of the coefficient matrix for each lag p>=1 and the corresponding stability
        prt = False: return the overall stability result for the lag p
        """
        table_header = ['p', 'Eigenvalues', 'Modulus', 'Stable']
        table_data = []
        stability = True
        for i in range(1, self.p + 1):
            for j in range(len(self.Eigenvalues[i])):
                val = self.Eigenvalues[i][j]
                if abs(val) >= 1:
                    stability = False
                if j==0:
                    table_data.append([str(i)] + ['{:.4f}'.format(val), '{:.4f}'.format(abs(val)), abs(val) < 1])
                else:
                    table_data.append([' '] + ['{:.4f}'.format(val), '{:.4f}'.format(abs(val)), abs(val) < 1])
        if prt:
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        else:      
            return stability
        
    def correlation(self):
        Rmat = self.res['resid']
        # get the resdiual correlation
        residual_corr = pd.DataFrame(Rmat.T, columns=self.col_names).corr()
        # return the pair correlation in descending order
        return residual_corr.stack().sort_values(ascending=False)[len(self.col_names)::2]
        

### Engle-Granger Procedure

The class **EG(X, Y)** construct an instance of the Engle-Granger procedure for variables X and Y, which consists of three parts:

- 1. Regression of Y based on X

- 2. Augmented Dickey Fuller Test on the residual of the regression

- 3. Error correction equation

**Inputs**:

- X: the explanatory variable in pd.DataFrame
- Y: the dependent variable in pd.DataFrame

**Methods**:

- OLS(prt): regress Y on X
    - prt = True: default case; print a table of result which contains information of coefficients, standard error, T statistic value and p value
    - prt = False: return the coefficients


- ADF_Test(prt): 
    - prt = True: default case; print a table of the ADF result which contains information of coefficients, standard error, T statistic value and p value
    - prt = False: return the dictionary containing the regression result


- error_correction(prt): 
    - prt = True: default case; print a table of the error correction regression result which contains information of coefficients, standard error, T statistic value and p value
    - prt = False: return the dictionary containing the regression result


This class aims to test the residual's stationarity and also find the best pair of futures for the cointegration trading strategy.


**Further information: the ADF_Test(prt) method used here**:
- including the constant
- with lag of 1
- not including the time dependent trend

In [6]:
class EG():
    def __init__(self, X, Y):
        """
        Augmented Dickey Fuller test with lag 1 
        Inputs:
            X: independent variable in DataFrame
            Y: dependent variable in DataFrame
        """
        self.X = X
        self.Y = Y
        # number of observations
        N = len(self.X)
        # Form the explanatory matrix
        Xmat = explanatory_matrix(self.X, 0)
        # Form the dependent matrix
        Ymat = self.Y.values.T
        # OLS result
        self.res = regress(Xmat, Ymat)
        self.coef = self.res['coef']
        self.e = pd.DataFrame(self.res['resid'], index=self.X.index) 
        
    def OLS(self, prt = True):
        """
        print the estimation result of regression
        """
        table_header = ['', 'Const']
        table_data = []
        for i in range(1, len(self.coef)):
            table_header += ['beta'+str(i)]
        table_data.append(['coef'] + list(self.coef))
        table_data.append(['std err'] + list(chain.from_iterable(self.res['std err'])))
        table_data.append(['t-stats'] + list(chain.from_iterable(self.res['t-stats'])))
        table_data.append(['p-value'] + list(chain.from_iterable(self.res['p-value'])))
        if prt:
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        else:
            return self.coef
    
    def ADF_Test(self, prt = True):
        """
        print the estimation result of ADF test
        """
        # delta residual
        delta_resid = self.e.diff().dropna()
        # residual of lag 1
        resid_lag = self.e.shift(1).dropna()
        # delta residual of lag 1
        delta_resid_lag = delta_resid.shift(1).dropna()
        
        adf_data = pd.concat([resid_lag, delta_resid_lag, delta_resid], axis=1).dropna()
        adf_data.columns = ['resid_lag','delta_resid_lag', 'delta_resid']
        Nobs = len(adf_data)
        
        # explanatory matrix(with const.) and dependent matrix
        X_resid = explanatory_matrix(adf_data[['resid_lag', 'delta_resid_lag']], 0)
        Y_resid = adf_data['delta_resid'].values.T

        result = regress(X_resid, Y_resid)
        
        table_header = ['', 'Const', 'φ', 'φ1']
        table_data = []
        table_data.append(['coef'] + list(result['coef']))
        table_data.append(['std err'] + list(chain.from_iterable(result['std err'])))
        table_data.append(['t-stats'] + list(chain.from_iterable(result['t-stats'])))
        table_data.append(['p-value'] + list(chain.from_iterable(result['p-value'])))
        if prt:
            print('Critical Values:','\n 1%: ', CriticalValue('1%',Nobs), '\n 5%: ', CriticalValue('5%',Nobs), '\n 10%: ', CriticalValue('10%',Nobs))
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        else:
            return result
        
    def error_correction(self, prt = True):
        """
        This function gives the results of estimating parameters and corresponding statistics
        """
        # change of variable X
        delta_X = self.X.diff().dropna()
        # change of variable Y
        delta_Y = self.Y.diff().dropna()
        # residual of lag 1
        resid_lag = self.e.shift(1).dropna()
        # number of observed samples
        Nobs = len(delta_X)
        # form the dependent matrix
        Ymat = delta_Y.values.T
        # form the explanatory matrix (no const.)
        Xmat = pd.concat([delta_X, resid_lag], axis=1).values.T
        
        result = regress(Xmat, Ymat)
        table_header = ['', 'φ', '-(1-α)']
        table_data = []
        table_data.append(['coef'] + list(result['coef']))
        table_data.append(['std err'] + list(chain.from_iterable(result['std err'])))
        table_data.append(['t-stats'] + list(chain.from_iterable(result['t-stats'])))
        table_data.append(['p-value'] + list(chain.from_iterable(result['p-value'])))
        if prt:
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        else:
            return result

### Ornstein - Uhlenbeck Process

The class **OU(resid)** construct an instance of the stationary residual's Ornstein - Uhlenbeck mean reversion rocess:

**Inputs**:

- resid: the stationary residual in pd.DataFrame

**Methods**:

- fit(prt): AR(1) regression on the residual
    - prt = True: default case; print the estimation result which contains information of coefficients, standard error, T statistic value and p value
    - prt = False: -


- reversion_parameters(prt): return the value of equilibrium $\mu_{e}$, standard deviation $\sigma_{eq}$ and halflife
    - prt = True: default case; print a table of the parameters of the fitted mean reverting process
    - prt = False: -


This class aims to fit the stationary residual to the OU process in order to get the parameters used in constructing the trading strategy.

In [7]:
class OU():
    def __init__(self, resid):
        """
        This class gives the estimation result of fitting the residual to the Ornstein - Uhlenbeck Process
        Inputs:
            resid: the stationary residual in pd.DataFrame
        """
        self.resid = resid
        
    def fit(self, prt = True):
        """
        Fitting the residual to AR(1)
        """
        # residual of lag 1
        resid_lag = self.resid.shift(1).dropna()
        # form the dependent matrix
        dep_matrix = self.resid[1:].values.T
        # form the explanatory matrix, explanatory_matrix() function used for add a const row
        exp_matrix = explanatory_matrix(resid_lag, 0)
        # fitting
        result = regress(exp_matrix, dep_matrix)
        # coefficients
        self.C = list(result['coef'])[0]
        self.B = list(result['coef'])[1]
        # SSE
        self.SSE = result['resid']@result['resid'].T
        
        table_header = ['', 'C', 'B']
        table_data = []
        table_data.append(['coef'] + list(result['coef']))
        table_data.append(['std err'] + list(chain.from_iterable(result['std err'])))
        table_data.append(['t-stats'] + list(chain.from_iterable(result['t-stats'])))
        table_data.append(['p-value'] + list(chain.from_iterable(result['p-value'])))
        if prt:
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        
    def reversion_parameters(self, prt=True):
        """
        calculate the mean reverting parameters and return the mu_e, sigma_eq and halflife in days
        """
        # tau
        tau = 1.0/len(self.resid)
        # theta parameter
        theta = - np.log(self.B)/tau
        # mu_e parameter
        mu_e = self.C/(1-self.B)
        # sigma_eq parameter
        sigma_eq = np.sqrt(self.SSE * tau/(1-np.exp(-2*theta*tau)))
        # sigma_ou parameter
        sigma_ou = sigma_eq*np.sqrt(2*theta)
        # halflife
        tau_tilde = np.log(2)/theta
        # halflife in days
        halflife = tau_tilde/tau
        
        table_header = ['Parameters', 'Values']
        table_data = [['θ', theta],
                      ['μe', mu_e],
                      ['σeq', sigma_eq],
                      ['σou', sigma_ou],
                      ['halflife', tau_tilde],
                      ['halflife(days)', halflife]]
        if prt:
            print(tabulate(table_data, headers=table_header, tablefmt='pipe'))
        return (mu_e, sigma_eq, halflife)

## Finding the Pair

In [8]:
data_prices = pd.read_csv('futures_train.csv', index_col='Date')
data_returns = data_prices.pct_change().dropna()

The lags for Vector Autoregression are set to be from 1 to 10

In [9]:
# check with lag of 1 to 10 (1 day to two weeks)
lags = list(range(1,11))
table_header = ['Lag', 'AIC', 'BIC', 'Stability']
table_data = []
for p in lags:
    var = VAR(data_returns, p)
    table_data.append(var.AIC_BIC(False) + [var.Stability(False)])
    del var
print(tabulate(table_data, headers=table_header, tablefmt='pipe'))

|   Lag |      AIC |      BIC | Stability   |
|------:|---------:|---------:|:------------|
|     1 | -145.842 | -145.275 | True        |
|     2 | -145.779 | -144.676 | True        |
|     3 | -145.696 | -144.056 | True        |
|     4 | -145.63  | -143.452 | True        |
|     5 | -145.594 | -142.879 | True        |
|     6 | -145.525 | -142.272 | True        |
|     7 | -145.456 | -141.665 | True        |
|     8 | -145.385 | -141.056 | True        |
|     9 | -145.332 | -140.465 | True        |
|    10 | -145.286 | -139.879 | True        |


Optimal lag p is determined by the lowest values of AIC, BIC statistics. From the table above, the lowest AIC and BIC values are obtained when the lag is 1. The stability condition is also satisfied. Next, I will show more information with the optimal lag (which is 1).

In [10]:
var = VAR(data_returns, 1)
var_estimation = var.estimation()
var_estimation

,Variable,Stats,Const,Cocoa(-1),Crude Oil(-1),Cotton(-1),Gold(-1),Feeder Cattle(-1),Lean Hogs(-1),Copper(-1),...,KC HRW Wheat(-1),Lumber(-1),Live Cattle(-1),Natural Gas(-1),Sugar(-1),Silver(-1),Corn(-1),Oat(-1),Rough Rice(-1),Soybean(-1)
0,Cocoa,Estimates,0.0002,0.0011,0.0052,0.0438,0.0414,0.0195,-0.0065,-0.0283,...,-0.0056,-0.0067,0.0097,0.0102,0.0036,-0.0122,0.0115,-0.0239,0.0373,0.0605
1,Cocoa,Std err,0.0003,0.0170,0.0151,0.0170,0.0441,0.0358,0.0143,0.0199,...,0.0205,0.0144,0.0321,0.0094,0.0149,0.0264,0.0225,0.0121,0.0191,0.0219
2,Cocoa,t-stats,0.6646,0.0665,0.3468,2.5744,0.9397,0.5448,-0.4554,-1.4258,...,-0.2753,-0.4682,0.3032,1.0775,0.2427,-0.4635,0.5098,-1.9724,1.9499,2.7583
3,Cocoa,p-value,0.5063,0.9470,0.7287,0.0100,0.3474,0.5859,0.6488,0.1539,...,0.7831,0.6397,0.7617,0.2813,0.8082,0.6430,0.6102,0.0486,0.0512,0.0058
4,Crude Oil,Estimates,0.0005,-0.0128,-0.0451,0.0144,-0.0216,-0.0143,-0.0183,-0.0008,...,0.0061,0.0180,0.0579,0.0107,0.0028,-0.0162,-0.0513,-0.0080,-0.0275,0.0796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Rough Rice,p-value,0.1104,0.7264,0.0159,0.0430,0.6793,0.3852,0.0098,0.8061,...,0.4565,0.4949,0.4078,0.8118,0.0165,0.9848,0.3190,0.9012,0.0110,0.0468
68,Soybean,Estimates,0.0004,-0.0013,-0.0314,0.0506,-0.0196,-0.0058,-0.0230,-0.0174,...,-0.0017,0.0030,0.0190,0.0069,0.0156,-0.0151,-0.0418,-0.0111,-0.0091,0.0603
69,Soybean,Std err,0.0003,0.0158,0.0140,0.0158,0.0409,0.0332,0.0133,0.0184,...,0.0190,0.0133,0.0298,0.0088,0.0138,0.0245,0.0209,0.0112,0.0177,0.0204
70,Soybean,t-stats,1.3100,-0.0796,-2.2456,3.2053,-0.4799,-0.1752,-1.7304,-0.9453,...,-0.0914,0.2228,0.6362,0.7907,1.1328,-0.6155,-2.0018,-0.9859,-0.5136,2.9599


In [11]:
var_t = var_estimation[var_estimation['Stats'] == 't-stats']
var_t.set_index('Variable', inplace=True, drop=True)
var_t = var_t.drop(['Stats', 'Const'], axis=1)
var_t

,Cocoa(-1),Crude Oil(-1),Cotton(-1),Gold(-1),Feeder Cattle(-1),Lean Hogs(-1),Copper(-1),Coffee(-1),KC HRW Wheat(-1),Lumber(-1),Live Cattle(-1),Natural Gas(-1),Sugar(-1),Silver(-1),Corn(-1),Oat(-1),Rough Rice(-1),Soybean(-1)
Variable,,,,,,,,,,,,,,,,,,
Cocoa,0.0665,0.3468,2.5744,0.9397,0.5448,-0.4554,-1.4258,0.8847,-0.2753,-0.4682,0.3032,1.0775,0.2427,-0.4635,0.5098,-1.9724,1.9499,2.7583
Crude Oil,-0.6084,-2.4210,0.6841,-0.3966,-0.3235,-1.0343,-0.0332,2.1872,0.2390,1.0144,1.4566,0.9207,0.1532,-0.4948,-1.8419,-0.5373,-1.1601,2.9292
Cotton,-0.0919,0.7461,4.5594,-1.3110,-2.3052,-2.3299,-1.7604,1.5892,-0.1881,-1.5329,1.3628,0.4195,-1.0734,0.6181,0.0000,-0.4766,-0.3005,0.8207
Gold,1.0108,0.1516,-0.0577,-0.2966,1.5457,1.0741,-2.2249,0.7168,1.3143,-1.8393,-1.4154,1.3510,0.8187,-0.0192,0.3422,-0.6768,2.0732,0.9194
Feeder Cattle,-0.4330,2.2983,-0.6226,0.3881,2.9374,0.0836,-2.1978,-2.4444,-0.0504,1.3911,2.3559,-1.3038,1.8578,-0.9342,-2.7288,-1.2742,0.4120,4.3795
Lean Hogs,-0.2972,1.5506,0.7348,-1.5964,-1.3521,1.8835,-1.4542,-0.4533,0.1543,0.7299,0.5942,-0.0384,0.8108,1.1116,-4.0076,0.8948,-1.1595,3.3573
Copper,-0.9769,-0.3793,-0.1102,-0.3175,0.1396,0.0153,-2.7018,0.6059,1.4069,-0.3241,0.1428,0.4674,0.4952,-1.1208,-2.0576,-1.5033,0.4273,5.7528
Coffee,-1.2335,-1.2043,0.8447,1.1434,1.4700,1.1623,-0.5428,-2.1744,1.5309,-0.6426,-0.5768,2.2638,-0.2959,-0.1894,-0.1737,1.1217,0.7264,1.6147
KC HRW Wheat,0.4691,-2.8100,2.7038,0.6634,0.6995,-0.3637,-1.2366,1.1028,2.4726,-0.6203,-1.5115,-0.2377,1.6093,-0.6315,-0.9140,-3.2474,-1.6380,2.4327


For this dataset, there are 18 different futures (18 variables) in total, so the Degree of Freedom is 17. The critical values of two-tail T distribution of 17 degree of freedom for 10%, 5% and 1% are 1.7396, 2.1098 and 2.8983 respectively. We need to check the t-stats confirm the significance of the lag 1 variable in VAR model. The significance level used is 1%.

- **Null Hypothesis $H_{0}$**: the coefficient is 0.

- **Alternative Hypothesis $H_{1}$**: the coefficient is not 0.

In [12]:
table_data = []
for row in var_t.index:
    for col in var_t.columns:
        if abs(float(var_t.loc[row, col])) > 2.8983:
            var_t.loc[row, col] = np.sign(float(var_t.loc[row, col])) 
            if np.sign(float(var_t.loc[row, col]))==1:
                table_data.append([row, col])
        else:
            var_t.loc[row, col] = 0
var_t

,Cocoa(-1),Crude Oil(-1),Cotton(-1),Gold(-1),Feeder Cattle(-1),Lean Hogs(-1),Copper(-1),Coffee(-1),KC HRW Wheat(-1),Lumber(-1),Live Cattle(-1),Natural Gas(-1),Sugar(-1),Silver(-1),Corn(-1),Oat(-1),Rough Rice(-1),Soybean(-1)
Variable,,,,,,,,,,,,,,,,,,
Cocoa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Crude Oil,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
Cotton,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Gold,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Feeder Cattle,0,0,0,0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
Lean Hogs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.0,0,0,1.0
Copper,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
Coffee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
KC HRW Wheat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.0,0,0


Select out the pairs with significant positive coefficient.

In [13]:
table_header = ['Variable', 'Lag(1) Variable']
print(tabulate(table_data, headers=table_header, tablefmt='pipe'))

| Variable      | Lag(1) Variable   |
|:--------------|:------------------|
| Crude Oil     | Soybean(-1)       |
| Cotton        | Cotton(-1)        |
| Feeder Cattle | Feeder Cattle(-1) |
| Feeder Cattle | Soybean(-1)       |
| Lean Hogs     | Soybean(-1)       |
| Copper        | Soybean(-1)       |
| Lumber        | Lumber(-1)        |
| Live Cattle   | Feeder Cattle(-1) |
| Live Cattle   | Soybean(-1)       |
| Soybean       | Cotton(-1)        |
| Soybean       | Soybean(-1)       |


The pairwise futures of each of pair in the above list are significantly positively correlated according to the VAR model.

In [14]:
var.Stability()

| p   |   Eigenvalues |   Modulus |   Stable |
|:----|--------------:|----------:|---------:|
| 1   |        0.1224 |    0.1224 |        1 |
|     |        0.09   |    0.09   |        1 |
|     |        0.088  |    0.088  |        1 |
|     |        0.088  |    0.088  |        1 |
|     |        0.0816 |    0.0816 |        1 |
|     |        0.0816 |    0.0816 |        1 |
|     |        0.0669 |    0.0669 |        1 |
|     |        0.0361 |    0.0361 |        1 |
|     |        0.0359 |    0.0359 |        1 |
|     |        0.0359 |    0.0359 |        1 |
|     |        0.0572 |    0.0572 |        1 |
|     |        0.0572 |    0.0572 |        1 |
|     |        0.0675 |    0.0675 |        1 |
|     |        0.0675 |    0.0675 |        1 |
|     |        0.0562 |    0.0562 |        1 |
|     |        0.0562 |    0.0562 |        1 |
|     |        0.037  |    0.037  |        1 |
|     |        0.0129 |    0.0129 |        1 |


In [15]:
var.correlation()

Gold           Silver           0.792950
KC HRW Wheat   Corn             0.582558
Corn           Soybean          0.546765
Feeder Cattle  Live Cattle      0.545642
Copper         Silver           0.452464
                                  ...   
Lean Hogs      Gold            -0.018117
Oat            Feeder Cattle   -0.026289
Live Cattle    Natural Gas     -0.027761
Gold           Feeder Cattle   -0.058467
Feeder Cattle  Corn            -0.071494
Length: 153, dtype: float64

In [16]:
top5 = [('Gold','Silver'), ('KC HRW Wheat','Corn'), ('Corn','Soybean'), ('Feeder Cattle','Live Cattle'), ('Copper','Silver')]

In [17]:
del var

The table above shows that the price returns of 'Gold' and 'Silver' future are highly correlated. We can see that the pair of assets which are of the same kind (eg. precious metal, cereals, livestock...) are higher correlated, which is quite intuitive and reasonable. However, high correlation does not mean the high cointegration. For the top 5 highest correlated pairs, Augmented Dickey Fuller test is used to test the level of cointegration.

In [18]:
for x, y in top5:
    print(x, ' - ', y)
    adf = EG(data_prices[x], data_prices[y])
    adf.ADF_Test()
    del adf
    print('\n')

Gold  -  Silver
Critical Values: 
 1%:  -3.8993512340213963 
 5%:  -3.3377537812697278 
 10%:  -3.0455769720073045
|         |       Const |           φ |        φ1 |
|:--------|------------:|------------:|----------:|
| coef    | -0.00154389 | -0.00337267 | 0.0299409 |
| std err |  0.00468862 |  0.00137355 | 0.0162973 |
| t-stats | -0.329285   | -2.45545    | 1.83716   |
| p-value |  0.74194    |  0.0140709  | 0.0661861 |


KC HRW Wheat  -  Corn
Critical Values: 
 1%:  -3.8993512340213963 
 5%:  -3.3377537812697278 
 10%:  -3.0455769720073045
|         |      Const |            φ |          φ1 |
|:--------|-----------:|-------------:|------------:|
| coef    | 0.00999056 | -0.00548272  | 0.109632    |
| std err | 0.138169   |  0.00161452  | 0.0162014   |
| t-stats | 0.072307   | -3.39587     | 6.76682     |
| p-value | 0.942358   |  0.000684104 | 1.31648e-11 |


Corn  -  Soybean
Critical Values: 
 1%:  -3.8993512340213963 
 5%:  -3.3377537812697278 
 10%:  -3.0455769720073045
|       

The important value is the t-stats of $\phi$ and its corresponding p value. From the results above we can see, among the top 5 highest correlated pairs, Gold-Silver and Copper-Silver pairs does not pass the ADF test, which says we are not significantly confident to reject the null hypothesis of that the residual of the pair has unit root.

KC HRW Wheat-Corn pair rejects the null hypothesis at 5% of significance level.

Corn-Soybean pair rejects the null hypothesis at 1% of significance level (although not that much).

The pair that rejects the null hypothesis with the highest confidence level should be **Feeder Cattle-Live Cattle** pair (with t-stats of -4.5112 and p value of 6.44613e-06). Thus, Feeder Cattle and Live Cattle are chosen for constructing the statistical arbitrage strategy of cointegretion in this project.

## Extra Workings with Johansen Procedure

As an extra confirmation of the selected pair of *Feeder Cattle* and *Live Cattle* as well as a further exploration of the multivariate cointegration relationships, the Johansen procedure is applied to the training dataset.

The function of *coint_johansen()* of the *statsmodels* package in python is used here, it has three input parameters[1]:

- **endog(array_like, nobs_tot x neqs)**: Data to test

- **det_order(int)**:
    - -1 - no deterministic terms

    - 0 - constant term

    - 1 - linear trend

- **k_ar_diff(int, non-negative)** Number of lagged differences in the model.


The function of *select_coint_rank()* of the *statsmodels* package in python is used here as well, the inputs are:

- **endog(array_like, nobs_tot x neqs)**: Data to test

- **det_order(int)**:
    - -1 - no deterministic terms

    - 0 - constant term

    - 1 - linear trend

- **k_ar_diff(int, non-negative)** Number of lagged differences in the model.

- **method(str)**: {"trace", "maxeig"}, default: "trace". If "trace", the trace test statistic is used. If "maxeig", the maximum eigenvalue test statistic is used.

- **signif(float)**: {0.1, 0.05, 0.01}, default: 0.05. The test’s significance level.


We set *det_orderi* to be 0 to include the constant term without the trend term. The optimal lag is 1 as shown before according to the AIC-BIC result of the VAR model. 

We check the selected pair *Feeder Cattle*-*Live Cattle* first.

In [19]:
import statsmodels.tsa.vector_ar.vecm as cajo

def johansen(data, det_order, k_ar_diff):
    # johansen test
    johansen_test = cajo.coint_johansen(data, det_order, k_ar_diff)
    # rank information of significance level of 1%
    rank = cajo.select_coint_rank(data, det_order, k_ar_diff, signif=0.01)
    
    # trace test
    j_lr1 = johansen_test.lr1
    j_cvt = johansen_test.cvt  # Critical values (90%, 95%, 99%) of trace statistic
    j_order = johansen_test.ind  # Order of eigenvalues
    j_name_trace=np.array([['Order'], ['Trace Stats'], ['Trace CV 90%'],  ['Trace CV 95%'], ['Trace CV 99%']])
    j_result_trace = np.vstack((j_order, j_lr1, j_cvt.T))
    j_result_trace = np.hstack((j_name_trace, j_result_trace))
    
    # maximum eigenvalue test
    j_lr2 = johansen_test.lr2
    j_cvm = johansen_test.cvm  # Critical values (90%, 95%, 99%) of max eigenvalue statistic
    j_name_maxeig=np.array([['Order'], ['Max_eig Stats'], ['Max_eig CV 90%'],  ['Max_eig CV 95%'], ['Max_eig CV 99%']])
    j_result_maxeig = np.vstack((j_order, j_lr2, j_cvm.T))
    j_result_maxeig = np.hstack((j_name_maxeig, j_result_maxeig))
    
    
    print('======Rank======')
    print(rank.summary())
    print('The rank of Π is ', rank.rank)
    print('\n')
    print('======Johansen Test Trace Test======')
    print(tabulate(j_result_trace, tablefmt='pipe'))
    print('\n')
    print('======Johansen Test Maximum Eigenvalue Test======')
    print(tabulate(j_result_maxeig, tablefmt='pipe'))
    

In [20]:
johansen(data_prices[['Feeder Cattle','Live Cattle']], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   2          22.44          19.93
  1   2          2.149          6.635
-------------------------------------
The rank of Π is  1


======Johansen Test Trace Test======
|:-------------|--------:|--------:|
| Order        |  0      | 1       |
| Trace Stats  | 22.4441 | 2.14927 |
| Trace CV 90% | 13.4294 | 2.7055  |
| Trace CV 95% | 15.4943 | 3.8415  |
| Trace CV 99% | 19.9349 | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|--------:|--------:|
| Order          |  0      | 1       |
| Max_eig Stats  | 20.2949 | 2.14927 |
| Max_eig CV 90% | 12.2971 | 2.7055  |
| Max_eig CV 95% | 14.2639 | 3.8415  |
| Max_eig CV 99% | 18.52   | 6.6349  |


Accoding to the result above, there is 1 cointegration pair. Both the trace test and maximum eigenvalue test show the cointegration relationship between *Feeder Cattle*-*Live Cattle* pair with significance level of 1%.

It is interesting to explore the cointegration relationship among the prices of the futures of the same category. So next, let's try to analysis on the cointegration relationship of the same-categorical futures based on Johansen test. 

###  Grain Futures 

In [21]:
grain = ['KC HRW Wheat', 'Corn', 'Oat', 'Rough Rice', 'Soybean']
johansen(data_prices[grain], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   5          120.1          77.82
  1   5          79.53          54.68
  2   5          43.99          35.46
  3   5          17.82          19.93
-------------------------------------
The rank of Π is  3


======Johansen Test Trace Test======
|:-------------|---------:|--------:|--------:|--------:|--------:|
| Order        |   0      |  1      |  2      |  3      | 4       |
| Trace Stats  | 120.141  | 79.5334 | 43.9919 | 17.8202 | 4.59003 |
| Trace CV 90% |  65.8202 | 44.4929 | 27.0669 | 13.4294 | 2.7055  |
| Trace CV 95% |  69.8189 | 47.8545 | 29.7961 | 15.4943 | 3.8415  |
| Trace CV 99% |  77.8202 | 54.6815 | 35.4628 | 19.9349 | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|--------:|--------:|--------:|--------:|--------:|
| Order          |  0      |  1      |  2      

There are 3 cointegration relationships among 5 grain futures at significant level of 1%.

###  Metals

In [22]:
metals = ['Gold', 'Silver', 'Copper']
johansen(data_prices[metals], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   3          19.12          35.46
-------------------------------------
The rank of Π is  0


======Johansen Test Trace Test======
|:-------------|--------:|---------:|--------:|
| Order        |  0      |  1       | 2       |
| Trace Stats  | 19.1183 |  8.26633 | 2.53842 |
| Trace CV 90% | 27.0669 | 13.4294  | 2.7055  |
| Trace CV 95% | 29.7961 | 15.4943  | 3.8415  |
| Trace CV 99% | 35.4628 | 19.9349  | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|--------:|---------:|--------:|
| Order          |  0      |  1       | 2       |
| Max_eig Stats  | 10.852  |  5.72791 | 2.53842 |
| Max_eig CV 90% | 18.8928 | 12.2971  | 2.7055  |
| Max_eig CV 95% | 21.1314 | 14.2639  | 3.8415  |
| Max_eig CV 99% | 25.865  | 18.52    | 6.6349  |


Despite of the high correlation according to the results above, there is no cointegration among the futures of 3 precious metals.

### Cash Crop

In [23]:
cash_crop = ['Cocoa', 'Coffee', 'Sugar', 'Cotton']
johansen(data_prices[cash_crop], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   4          67.22          54.68
  1   4          35.27          35.46
-------------------------------------
The rank of Π is  1


======Johansen Test Trace Test======
|:-------------|--------:|--------:|--------:|--------:|
| Order        |  0      |  1      |  2      | 3       |
| Trace Stats  | 67.2164 | 35.2654 | 14.0237 | 4.53296 |
| Trace CV 90% | 44.4929 | 27.0669 | 13.4294 | 2.7055  |
| Trace CV 95% | 47.8545 | 29.7961 | 15.4943 | 3.8415  |
| Trace CV 99% | 54.6815 | 35.4628 | 19.9349 | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|--------:|--------:|---------:|--------:|
| Order          |  0      |  1      |  2       | 3       |
| Max_eig Stats  | 31.9511 | 21.2416 |  9.49075 | 4.53296 |
| Max_eig CV 90% | 25.1236 | 18.8928 | 12.2971  | 2.7055  |
| Max_eig CV 95% |

There is one cointegration relationship among cash corp futures in our dataset.

### Livestock

In [24]:
livestock = ['Feeder Cattle','Live Cattle','Lean Hogs']
johansen(data_prices[livestock], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   3          72.74          35.46
  1   3          32.96          19.93
  2   3          3.443          6.635
-------------------------------------
The rank of Π is  2


======Johansen Test Trace Test======
|:-------------|--------:|--------:|--------:|
| Order        |  0      |  1      | 2       |
| Trace Stats  | 72.7446 | 32.9609 | 3.44275 |
| Trace CV 90% | 27.0669 | 13.4294 | 2.7055  |
| Trace CV 95% | 29.7961 | 15.4943 | 3.8415  |
| Trace CV 99% | 35.4628 | 19.9349 | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|--------:|--------:|--------:|
| Order          |  0      |  1      | 2       |
| Max_eig Stats  | 39.7837 | 29.5181 | 3.44275 |
| Max_eig CV 90% | 18.8928 | 12.2971 | 2.7055  |
| Max_eig CV 95% | 21.1314 | 14.2639 | 3.8415  |
| Max_eig CV 99% | 25.865  | 18.52 

There are two cointegration relationship among 3 livestock futures.

### Energy

In [25]:
energy = ['Crude Oil', 'Natural Gas']
johansen(data_prices[energy], 0, 1)

======Rank======
Johansen cointegration test using trace test statistic with 1% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   2          11.52          19.93
-------------------------------------
The rank of Π is  0


======Johansen Test Trace Test======
|:-------------|--------:|--------:|
| Order        |  0      | 1       |
| Trace Stats  | 11.5196 | 4.30315 |
| Trace CV 90% | 13.4294 | 2.7055  |
| Trace CV 95% | 15.4943 | 3.8415  |
| Trace CV 99% | 19.9349 | 6.6349  |


======Johansen Test Maximum Eigenvalue Test======
|:---------------|---------:|--------:|
| Order          |  0       | 1       |
| Max_eig Stats  |  7.21643 | 4.30315 |
| Max_eig CV 90% | 12.2971  | 2.7055  |
| Max_eig CV 95% | 14.2639  | 3.8415  |
| Max_eig CV 99% | 18.52    | 6.6349  |


No cointegration between *Crude Oil* and *Natural Gas*.

# Reference

[1] https://www.statsmodels.org/stable/generated/statsmodels.tsa.vector_ar.vecm.coint_johansen.html